### Classification example using the MNIST dataset

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms # for preprocessing the images
from torch.utils.data import DataLoader

In [ ]:
# Dataset creation and importation from torchvision.datasets

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)) # normalize pixels for having a mean of 0.5 with a standard deviation of also 0.5
]) # it improves the learning phase !

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

100%|██████████| 9.91M/9.91M [00:03<00:00, 3.19MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 297kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 2.29MB/s]
100%|██████████| 4.54k/4.54k [00:00<?, ?B/s]


In [4]:
# Creation of the model, creating it as a class as it is recommended
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Flatten(), # transforms the 2D image to a 1D vector
            nn.Linear(28*28, 256), # first hidden layer
            nn.ReLU(), # activation function, set negative values to 0
            nn.Linear(256, 64), # second hidden layer
            nn.ReLU(), # activation function
            nn.Linear(64, 10) # output layer, 10 classes as there are 10 numbers
        )

    def forward(self, x):
        return self.net(x)
    
model = MLP()

In [5]:
# Optimizer and loss
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(3):
    for x, y in train_loader:
        logits = model(x) # raw predictions
        loss = criterion(logits, y)        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch}, Loss = {loss.item():.4f}")        

Epoch 0, Loss = 0.1190
Epoch 1, Loss = 0.2491
Epoch 2, Loss = 0.2511


In [7]:
# Testiiiiiiing
correct = 0
total = 0
with torch.no_grad():
    for x, y in test_loader:
        preds = model(x).argmax(1)
        correct += (preds == y).sum().item()
        total += y.size(0)
print(f"Accuracy = {100*correct/total:.2f}%")        

Accuracy = 96.67%
